In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [2]:
data = pd.read_csv('../input/train.csv',usecols=['title','description'])

![Word2Vec and FastText Word Embedding with Gensim](https://cdn-images-1.medium.com/max/2000/1*r_R38HJ9NkbPyb0Y7PkHRw.png)

In [3]:
data.head()

## Word2Vec
Word2Vec is an efficient solution to these problems, which leverages the context of the target words. Essentially, we want to use the surrounding words to represent the target words with a Neural Network whose hidden layer encodes the word representation.

There are two types of Word2Vec, Skip-gram and Continuous Bag of Words (CBOW). I will briefly describe how these two methods work in the following paragraphs.

### Skip-gram
For skip-gram, the input is the target word, while the outputs are the words surrounding the target words. For instance, in the sentence “I have a cute dog”, the input would be “a”, whereas the output is “I”, “have”, “cute”, and “dog”, assuming the window size is 5. All the input and output data are of the same dimension and one-hot encoded. The network contains 1 hidden layer whose dimension is equal to the embedding size, which is smaller than the input/ output vector size. At the end of the output layer, a softmax activation function is applied so that each element of the output vector describes how likely a specific word will appear in the context. The graph below visualizes the network structure.

![Skip-gram](https://cdn-images-1.medium.com/max/1600/1*TbjQNQLuyEW-cgsofyDioQ.png)

The word embedding for the target words can obtained by extracting the hidden layers after feeding the one-hot representation of that word into the network.

With skip-gram, the representation dimension decreases from the vocabulary size (V) to the length of the hidden layer (N). Furthermore, the vectors are more “meaningful” in terms of describing the relationship between words. The vectors obtained by subtracting two related words sometimes express a meaningful concept such as gender or verb tense, as shown in the following figure (dimensionality reduced).

![Visualize Word Vectors](https://cdn-images-1.medium.com/max/1600/1*jpnKO5X0Ii8PVdQYFO2z1Q.png)

### CBOW
Continuous Bag of Words (CBOW) is very similar to skip-gram, except that it swaps the input and output. The idea is that given a context, we want to know which word is most likely to appear in it.

![CBOW](https://cdn-images-1.medium.com/max/1600/1*UdLFo8hgsX0a1NKKuf_n9Q.png)

The biggest difference between Skip-gram and CBOW is that the way the word vectors are generated. For CBOW, all the examples with the target word as target are fed into the networks, and taking the average of the extracted hidden layer. For example, assume we only have two sentences, “He is a nice guy” and “She is a wise queen”. To compute the word representation for the word “a”, we need to feed in these two examples, “He is nice guy”, and “She is wise queen” into the Neural Network and take the average of the value in the hidden layer. Skip-gram only feed in the one and only one target word one-hot vector as input.

It is claimed that Skip-gram tends to do better in rare words. Nevertheless, the performance of Skip-gram and CBOW are generally similar.

In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import re

import pickle 
#import mglearn
import time


from nltk.tokenize import TweetTokenizer # doesn't split at apostrophes
import nltk
from nltk import Text
from nltk.tokenize import regexp_tokenize
from nltk.tokenize import word_tokenize  
from nltk.tokenize import sent_tokenize 
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer


from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [13]:
data.description.values[:10]

In [14]:
txt = ['Кокон для сна малыша,пользовались меньше месяца.цвет серый',
       'Стойка для одежды, под вешалки. С бутика.',
       'В хорошем состоянии, домашний кинотеатр с blu ray, USB. Если настроить, то работает смарт тв /\nТорг',
       'Продам кресло от0-25кг', 'Все вопросы по телефону.',
       'В хорошем состоянии',
       'Электро водонагреватель накопительный на 100 литров Термекс ID 100V, плоский, внутренний бак из нержавейки, 2 кВт, б/у 2 недели, на гарантии.',
       'Бойфренды в хорошем состоянии.', '54 раз мер очень удобное',
       'По стельке 15.5см мерить приокский район. Цвет темнее чем на фото']

In [17]:
# Initialize a CountVectorizer object: count_vectorizer
count_vec = CountVectorizer(stop_words=stopwords.words('russian'), analyzer='word', encoding='KOI8-R',
                            ngram_range=(1, 1), max_df=1.0, min_df=1, max_features=None)

# Transforms the data into a bag of words
count_train = count_vec.fit(txt)
bag_of_words = count_vec.transform(txt)

# Print the first 10 features of the count_vec
print("Every feature:\n{}".format(count_vec.get_feature_names()))
print("\nEvery 3rd feature:\n{}".format(count_vec.get_feature_names()[::3]))

In [18]:
print("Vocabulary size: {}".format(len(count_train.vocabulary_)))
print("Vocabulary content:\n {}".format(count_train.vocabulary_))

### N-grams (sets of consecutive words)
**N==2**

In [19]:
count_vec = CountVectorizer(stop_words=stopwords.words('russian'), analyzer='word', encoding='KOI8-R',
                            ngram_range=(1, 2), max_df=1.0, min_df=1, max_features=None)

count_train = count_vec.fit(txt)
bag_of_words = count_vec.transform(txt)

print(count_vec.get_feature_names())

**N==3**

In [20]:
count_vec = CountVectorizer(stop_words=stopwords.words('russian'), analyzer='word', encoding='KOI8-R',
                            ngram_range=(1, 3), max_df=1.0, min_df=1, max_features=None)

count_train = count_vec.fit(txt)
bag_of_words = count_vec.transform(txt)

print(count_vec.get_feature_names())

### Min_df

**Min_df ignores terms that have a document frequency (presence in % of documents) strictly lower than the given threshold. For example, Min_df=0.66 requires that a term appear in 66% of the docuemnts for it to be considered part of the vocabulary.**

Sometimes min_df is used to limit the vocabulary size, so it learns only those terms that appear in at least 10%, 20%, etc. of the documents.

In [24]:
count_vec = CountVectorizer(stop_words=stopwords.words('russian'), analyzer='word', encoding='KOI8-R',
                            ngram_range=(1, 1), max_df=1.0, min_df=0.3, max_features=None)

count_train = count_vec.fit(txt)
bag_of_words = count_vec.transform(txt)

print(count_vec.get_feature_names())
print("\nOnly 'park' becomes the vocabulary of the document term matrix (dtm) because it appears in 2 out of 3 documents, \
meaning 0.66% of the time.\
      \nThe rest of the words such as 'big' appear only in 1 out of 3 documents, meaning 0.33%. which is why they don't appear")

### Max_df
When building the vocabulary, it ignores terms that have a document frequency strictly higher than the given threshold. This could be used to exclude terms that are too frequent and are unlikely to help predict the label. For example, by analyzing reviews on the movie Lion King, the term 'Lion' might appear in 90% of the reviews (documents), in which case, we could consider establishing Max_df=0.89

In [25]:
count_vec = CountVectorizer(stop_words=stopwords.words('russian'), analyzer='word', encoding='KOI8-R',
                            ngram_range=(1, 1), max_df=0.50, min_df=1, max_features=None)

count_train = count_vec.fit(txt)
bag_of_words = count_vec.transform(txt)

print(count_vec.get_feature_names())
print("\nOnly 'park' is ignored because it appears in 2 out of 3 documents, meaning 0.66% of the time.")

### Max_features
Limit the amount of features (vocabulary) that the vectorizer will learn

In [26]:
count_vec = CountVectorizer(stop_words=stopwords.words('russian'), analyzer='word', encoding='KOI8-R',
                            ngram_range=(1, 1), max_df=1.0, min_df=1, max_features=4)

count_train = count_vec.fit(txt)
bag_of_words = count_vec.transform(txt)

print(count_vec.get_feature_names())

### TfidfVectorizer -- Brief Tutorial
The goal of using tf-idf is to scale down the impact of tokens that occur very frequently in a given corpus and that are hence empirically less informative than features that occur in a small fraction of the training corpus. (https://github.com/scikit-learn/scikit-learn/blob/a24c8b46/sklearn/feature_extraction/text.py#L1365)

In [27]:
data.description.values[10:30]

In [31]:
txt1 = ['Семейная пара из двух человек снимет коттедж с дизайнерским ремонтом, со стильной мебелью и бытовой техникой. Гараж на 2 машины, 2 санузла. Огороженная территория, сигнализация.  Рассмотрим варианты коттеджей, которые выставлены на продажу. Ежемесячная оплата до 100 тыс.руб. в месяц.  Агентства просьба не беспокоить.',
       'Дом находиться внутри квартала./\nПластиковые окна во всей квартире, балкон застеклен железом, обшит деревом, в комнате натяжной потолок, новый линолиум,/\nванна и туалет- стеновые панели.Квартира подходит под ипотеку, 1 собственник, ЧП, небольшой торг возможен.']
tf = TfidfVectorizer(smooth_idf=False, sublinear_tf=False, norm=None, analyzer='word',
                     stop_words=stopwords.words('russian'), encoding='KOI8-R')
txt_fitted = tf.fit(txt1)
txt_transformed = txt_fitted.transform(txt1)
print ("The text: ", txt1)
print ("The txt_transformed: ", txt_transformed)

The learned corpus vocabulary

In [32]:
tf.vocabulary_

IDF: The inverse document frequency

In [33]:
idf = tf.idf_
print(dict(zip(txt_fitted.get_feature_names(), idf)))
print("\nWe see that the tokens 'sang','she' have the most idf weight because \
they are the only tokens that appear in one document only.")
print("\nThe token 'not' appears 6 times but it is also in all documents, so its idf is the lowest")

Graphing inverse document frequency

In [34]:
rr = dict(zip(txt_fitted.get_feature_names(), idf))

In [39]:
token_weight = pd.DataFrame.from_dict(rr, orient='index').reset_index()
token_weight.columns=('token','weight')
token_weight = token_weight.sort_values(by='weight', ascending=False)[:10]
token_weight

sns.barplot(x='token', y='weight', data=token_weight)
plt.title("Inverse Document Frequency(idf) per token")
fig=plt.gcf()
fig.set_size_inches(10,5)
fig.set_figwidth(10)
plt.show()

Listing (instead of graphing) inverse document frequency

In [40]:
# get feature names
feature_names = np.array(tf.get_feature_names())
sorted_by_idf = np.argsort(tf.idf_)
print("Features with lowest idf:\n{}".format(
       feature_names[sorted_by_idf[:3]]))
print("\nFeatures with highest idf:\n{}".format(
       feature_names[sorted_by_idf[-3:]]))

Weight of tokens per document

In [47]:
data.description.values[132]

In [48]:
print("The token 'not' has  the largest weight in document #2 because it appears 3 times there. But in document #1\
 its weight is 0 because it does not appear there.")
txt_transformed.toarray()

TF-IDF - Maximum token value throughout the whole dataset

In [49]:
new1 = tf.transform(txt1)

# find maximum value for each of the features over all of dataset:
max_val = new1.max(axis=0).toarray().ravel()

#sort weights from smallest to biggest and extract their indices 
sort_by_tfidf = max_val.argsort()

print("Features with lowest tfidf:\n{}".format(
      feature_names[sort_by_tfidf[:3]]))

print("\nFeatures with highest tfidf: \n{}".format(
      feature_names[sort_by_tfidf[-3:]]))

MORE IS COMING...